In [1]:
!pip install ipytest pyspark > /dev/null 2>&1

In [2]:
import ipytest

ipytest.autoconfig()

# Descrição do desafio:

Suponha que você tenha um dataframe com as seguintes colunas:

- `Data`: data da dívida (formato "yyyy-MM-dd")
- `Devedor`: nome do devedor
- `Credor`: nome do credor
- `Valor`: valor da dívida

Seu desafio é criar um novo dataframe que contenha as seguintes informações:

- O valor total das dívidas por devedor, ordenado em ordem decrescente de dívida
- O valor total das dívidas por credor, ordenado em ordem decrescente de dívida
- O valor total das dívidas por mês (formato "MM/yyyy"), ordenado em ordem crescente de data

Como bônus, um dataframe com as seguintes informações:

- `Devedor`: nome do devedor
- `Credor`: nome do credor
- `Data`: data da dívida (formato "MM/yyyy")
- `Divida`: valor total das dívidas do mês

In [74]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc, asc, date_format

spark = SparkSession.builder.appName("Desafio DataFrame").getOrCreate()

data = [("2021-01-01", "Devedor A", "Credor 1", 100.0),
        ("2021-01-01", "Devedor B", "Credor 1", 50.0),
        ("2021-02-02", "Devedor C", "Credor 2", 75.0),
        ("2021-02-02", "Devedor D", "Credor 2", 125.0),
        ("2021-03-03", "Devedor A", "Credor 1", 200.0),
        ("2021-03-03", "Devedor B", "Credor 1", 150.0),
        ("2021-04-04", "Devedor C", "Credor 2", 100.0),
        ("2021-04-04", "Devedor D", "Credor 2", 50.0)]

df = spark.createDataFrame(data, ["Data", "Devedor", "Credor", "Valor"])

In [7]:
df.show(10, False)

+----------+---------+--------+-----+
|Data      |Devedor  |Credor  |Valor|
+----------+---------+--------+-----+
|2021-01-01|Devedor A|Credor 1|100.0|
|2021-01-01|Devedor B|Credor 1|50.0 |
|2021-02-02|Devedor C|Credor 2|75.0 |
|2021-02-02|Devedor D|Credor 2|125.0|
|2021-03-03|Devedor A|Credor 1|200.0|
|2021-03-03|Devedor B|Credor 1|150.0|
|2021-04-04|Devedor C|Credor 2|100.0|
|2021-04-04|Devedor D|Credor 2|50.0 |
+----------+---------+--------+-----+



In [21]:
from pyspark.sql import functions as f

# **PySpark**

In [56]:
# Valor total das dívidas por devedor, ordenado em ordem decrescente de dívida
df_devedor = df.groupBy('Devedor')\
               .agg(f.sum('Valor').alias('Divida'))\
               .orderBy(col("Divida").desc())

df_devedor.show()

# Valor total das dívidas por credor, ordenado em ordem decrescente de dívida
df_credor = df.groupBy('Credor')\
               .agg(f.sum('Valor').alias('Divida'))\
               .orderBy(col("Divida").desc())

df_credor.show()

# Valor total das dívidas por mês no formato "MM/yyyy", ordenado em ordem crescente de data
df_mes = df.groupBy(date_format('Data', 'MM/yyyy').alias('Mes'))\
               .agg(f.sum('Valor').alias('Divida'))\
               .orderBy(col("Mes").asc())

df_mes.show()

+---------+------+
|  Devedor|Divida|
+---------+------+
|Devedor A| 300.0|
|Devedor B| 200.0|
|Devedor D| 175.0|
|Devedor C| 175.0|
+---------+------+

+--------+------+
|  Credor|Divida|
+--------+------+
|Credor 1| 500.0|
|Credor 2| 350.0|
+--------+------+

+-------+------+
|    Mes|Divida|
+-------+------+
|01/2021| 150.0|
|02/2021| 200.0|
|03/2021| 350.0|
|04/2021| 150.0|
+-------+------+



In [118]:
# Como bônus, um dataframe com as seguintes informações:

# Devedor: nome do devedor
# Credor: nome do credor
# Data: data da dívida (formato "MM/yyyy")
# Divida: valor total das dívidas do mês

# Bônus: dataframe com informações detalhadas
df_detalhado = df.select(f.col('Devedor'),\
                          f.col('Credor'),\
                          date_format('Data', "MM/yyyy").alias('Mes'),\
                          f.col('Valor').alias('Valor'))\
               .withColumn("Divida", f.col('Valor'))

df_detalhado = df_detalhado.drop('Valor')

df_detalhado.show()

+---------+--------+-------+------+
|  Devedor|  Credor|    Mes|Divida|
+---------+--------+-------+------+
|Devedor A|Credor 1|01/2021| 100.0|
|Devedor B|Credor 1|01/2021|  50.0|
|Devedor C|Credor 2|02/2021|  75.0|
|Devedor D|Credor 2|02/2021| 125.0|
|Devedor A|Credor 1|03/2021| 200.0|
|Devedor B|Credor 1|03/2021| 150.0|
|Devedor C|Credor 2|04/2021| 100.0|
|Devedor D|Credor 2|04/2021|  50.0|
+---------+--------+-------+------+



In [119]:
%%ipytest

def test_df_devedor():
    expected = [("Devedor A", 300.0),
                ("Devedor B", 200.0),
                ("Devedor D", 175.0),
                ("Devedor C", 175.0)]
    result = [(row.Devedor, row.Divida) for row in df_devedor.collect()]
    assert result == expected

def test_df_credor():
    expected = [("Credor 1", 500.0),
                ("Credor 2", 350.0)]
    result = [(row.Credor, row.Divida) for row in df_credor.collect()]
    assert result == expected

def test_df_mes():
    expected = [("01/2021", 150.0), ("02/2021", 200.0), ("03/2021", 350.0), ("04/2021", 150.0)]
    result = [(row.Mes, row.Divida) for row in df_mes.collect()]
    assert result == expected

def test_df_detalhado():
    expected = [("Devedor A", "Credor 1", "01/2021", 100.0),
                ("Devedor B", "Credor 1", "01/2021", 50.0),
                ("Devedor C", "Credor 2", "02/2021", 75.0),
                ("Devedor D", "Credor 2", "02/2021", 125.0),
                ("Devedor A", "Credor 1", "03/2021", 200.0),
                ("Devedor B", "Credor 1", "03/2021", 150.0),
                ("Devedor C", "Credor 2", "04/2021", 100.0),
                ("Devedor D", "Credor 2", "04/2021", 50.0)]
    result = [(row.Devedor, row.Credor, row.Mes, row.Divida) for row in df_detalhado.collect()]
    assert result == expected

....                                                                                         [100%]
4 passed in 0.44s
